$$\textbf{PLN. Tarea 3: Minería de texto básica}$$
$$\textit{Y. Sarahi García Gozález}$$

<font size=4.5 color='lightblue'>

$\textit{Librerías}$

In [4]:
from nltk.tokenize import TweetTokenizer
import nltk
import numpy as np
import pandas as pd
#clasificacion
import sklearn
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_recall_fscore_support,roc_auc_score

In [5]:
print("Tarea realizada en MacOs. \nLas versiones de las librerías y de python utilizadas fueron:\n")
from platform import python_version
print("Python version:", python_version())
print("NumPy version:", np.__version__)
print("NLTK version:", nltk.__version__)
print("Pandas version:", pd.__version__)
print("Scikit-learn version:", sklearn.__version__)

Tarea realizada en MacOs. 
Las versiones de las librerías y de python utilizadas fueron:

Python version: 3.11.5
NumPy version: 1.23.5
NLTK version: 3.8.1
Pandas version: 2.1.4
Scikit-learn version: 1.3.0




<font size=4.5 color='lightblue'>

$\textit{Selección de Términos y DTRs}$

* TCOR

* Random Indexing.

* Clasificación de
documentos 

* 10 pares de palabras más parecidas en toda la colección (TCOR)

*  10 pares de palabras más parecidas en toda la colección (BoW-TFIDF)

* 50 de las palabras más relevantes de TODA la
colección (Chi-2)